In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [40]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/clean_data.csv')
df.head()

,Unnamed: 0,reviews,ratings,new_ratings
0,0,This laptop is unbelievably fast. Between the ...,100%;,5
1,1,This Mac like all other Apple products arrived...,100%;,5
2,2,"Arrived fast, excellent memory RAM 2GB DDR 2, ...",100%;,5
3,3,I'm coming pretty fast.,100%;,5
4,4,"ALL COMBS FUNCTIONING PERFECTLY, ACCORDING TO ...",100%;,5


In [41]:
# create a function
def page_authenticity(new_ratings):
    if new_ratings >= 4:
        return "Authentic"
    elif new_ratings <3 or new_ratings ==2.5:
        return "Normal"
    else:
      return "Not Authentic"
# create a new column based on condition
df['page_authenticity'] = df['new_ratings'].apply(page_authenticity)
# display the dataframe
df.head()

,Unnamed: 0,reviews,ratings,new_ratings,page_authenticity
0,0,This laptop is unbelievably fast. Between the ...,100%;,5,Authentic
1,1,This Mac like all other Apple products arrived...,100%;,5,Authentic
2,2,"Arrived fast, excellent memory RAM 2GB DDR 2, ...",100%;,5,Authentic
3,3,I'm coming pretty fast.,100%;,5,Authentic
4,4,"ALL COMBS FUNCTIONING PERFECTLY, ACCORDING TO ...",100%;,5,Authentic


In [42]:
new_dataset= df.to_csv("new_dataset.csv", index=False)


Data Preparation

In [43]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [44]:
new_ratings_reviews=[]
for a, b in zip(df['reviews'].to_list(), df['new_ratings'].to_list()):
  new_ratings_reviews.append(str(a) + ' '+ str(b))




In [45]:
df['new_ratings_reviews']=new_ratings_reviews

In [46]:
df=df[['new_ratings_reviews', "page_authenticity"]]

In [47]:
"Drop duplicate values"
df.drop_duplicates(inplace=True)

In [48]:
df = df.dropna()

In [49]:
df.reset_index().rename(columns={'index':'u'}).drop('u', axis=1, inplace=True)

In [50]:
df.shape

(8368, 2)

In [51]:
df['page_authenticity'].replace('Authentic', 1, inplace=True)
df['page_authenticity'].replace('Normal', 2, inplace=True)
df['page_authenticity'].replace('Not Authentic', 3, inplace=True)

In [ ]:
df=df.reset_index()
df=df[['new_ratings_reviews',	'page_authenticity']]

In [ ]:
df['new_ratings_reviews'].to_list()[77]

In [61]:
train_x=df.loc[:7500, 'new_ratings_reviews']
train_y=df.loc[:7500, 'page_authenticity']
test_x= df.loc[7500:, 'new_ratings_reviews']
test_y= df.loc[7500:, 'page_authenticity']

In [62]:
num_words=3000
tokenizer=Tokenizer(num_words=num_words,oov_token='oov')
tokenizer.fit_on_texts(train_x.values)

In [63]:
tok_tr=tokenizer.texts_to_sequences(train_x.values)
tok_te=tokenizer.texts_to_sequences(test_x.values)

In [17]:
max_count=[]
for i in tok_tr:
  x=len(i)
  max_count.append(x)
max_count.sort()
max_count[-600]

11

In [64]:
pad_tr=pad_sequences(tok_tr,maxlen=10,padding='post', truncating='post')
pad_te=pad_sequences(tok_te,maxlen=10,padding='post', truncating='post')

Model Training for Rating Prediction Based on Review

In [65]:
model=tf.keras.Sequential([
                           tf.keras.layers.Embedding(input_dim=num_words,output_dim=64,input_length=10),
                           tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
                           tf.keras.layers.Dropout(0.8),
                           tf.keras.layers.Dense(16,activation='relu'),
                           tf.keras.layers.Dense(4, activation='softmax')]) 

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [68]:
test_y.unique()

array([1, 2, 3])

In [69]:
train_y=np.array(train_y)
test_y=np.array(test_y)


In [70]:
model.fit(pad_tr, train_y,validation_data=[pad_te, test_y],  epochs=6)

Epoch 1/6
235/235 [==============================] - 5s 10ms/step - loss: 0.6275 - accuracy: 0.7934 - val_loss: 0.1689 - val_accuracy: 0.9171
Epoch 2/6
235/235 [==============================] - 2s 7ms/step - loss: 0.2620 - accuracy: 0.8923 - val_loss: 0.1362 - val_accuracy: 0.9171
Epoch 3/6
235/235 [==============================] - 2s 7ms/step - loss: 0.1664 - accuracy: 0.9319 - val_loss: 0.0662 - val_accuracy: 0.9885
Epoch 4/6
235/235 [==============================] - 2s 6ms/step - loss: 0.0830 - accuracy: 0.9749 - val_loss: 0.0758 - val_accuracy: 0.9850
Epoch 5/6
235/235 [==============================] - 2s 6ms/step - loss: 0.0550 - accuracy: 0.9835 - val_loss: 0.0808 - val_accuracy: 0.9781
Epoch 6/6
235/235 [==============================] - 2s 6ms/step - loss: 0.0401 - accuracy: 0.9872 - val_loss: 0.0680 - val_accuracy: 0.9839


Prediction

In [80]:
pred='great laptop'
pred=tokenizer.texts_to_sequences([pred])



In [81]:
pred=pad_sequences(pred,maxlen=10,padding='post', truncating='post')

In [82]:
pred

array([[ 4, 24,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [83]:
import numpy as np
np.argmax(model.predict(pred))

1